## Overview

This Jupyter notebook makes it easy to :

1. Get the dataset and column metadata programmatically
2. Load CSV files automatically into a pandas dataframe so you can do the fun explorations

# Setup
1. Paste the dataset ID you copied into the cell below
2. Run All Cells (click `Runtime` -> `Run All`)

In [2]:
DATASET_ID = "PASTE_DATASET_ID_HERE" # e.g. "d_69b3380ad7e51aff3a7dcc84eba52b8a"

## Dataset and Column Metadata

In [3]:
import json
import requests

s = requests.Session()
s.headers.update({'referer': 'https://colab.research.google.com'})
base_url = "https://api-production.data.gov.sg"
url = base_url + f"/v2/public/api/datasets/{DATASET_ID}/metadata"
print(url)
response = s.get(url)
data = response.json()['data']
columnMetadata = data.pop('columnMetadata', None)

print("Dataset Metadata:")
print(json.dumps(data, indent=2))

print("\nColumns:\n", list(columnMetadata['map'].values()))


https://api-production.data.gov.sg/v2/public/api/datasets/PASTE_DATASET_ID_HERE/metadata


KeyError: 'data'

## Download File

In [ ]:
import time
import pandas as pd

def download_file(DATASET_ID):
  # initiate download
  initiate_download_response = s.get(
      f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/initiate-download",
      headers={"Content-Type":"application/json"},
      json={}
  )
  print(initiate_download_response.json()['data']['message'])

  # poll download
  MAX_POLLS = 5
  for i in range(MAX_POLLS):
    poll_download_response = s.get(
        f"https://api-open.data.gov.sg/v1/public/api/datasets/{DATASET_ID}/poll-download",
        headers={"Content-Type":"application/json"},
        json={}
    )
    print("Poll download response:", poll_download_response.json())
    if "url" in poll_download_response.json()['data']:
      print(poll_download_response.json()['data']['url'])
      DOWNLOAD_URL = poll_download_response.json()['data']['url']
      df = pd.read_csv(DOWNLOAD_URL)

      display(df.head())
      print("\nDataframe loaded!")
      return df
    if i == MAX_POLLS - 1:
      print(f"{i+1}/{MAX_POLLS}: No result found, possible error with dataset, please try again or let us know at https://go.gov.sg/datagov-supportform\n")
    else:
      print(f"{i+1}/{MAX_POLLS}: No result yet, continuing to poll\n")
    time.sleep(3)

df = download_file(DATASET_ID)


In [ ]:
df.describe()

In [ ]:
df.info()